# Learning Objectives


- Practice with http.server Python's module.
- Use encode() to send data from the server.
- Explain queries and quoting as a way to send data in a request.
- Differentiate a GET Request and a POST Request.
- Demonstrate Post-Redirect-Get
- Define JSON

# 

<img src='../images/' />

<img src='../images/' />

# 

<img src='../images/' />

<img src='../images/' />

# 

<img src='../images/' />

<img src='../images/' />

# 

<img src='../images/' />

<img src='../images/' />

# 

<img src='../images/' />

<img src='../images/' />

# 

<img src='../images/' />

<img src='../images/' />

# 

<img src='../images/' />

<img src='../images/' />

# 

<img src='../images/' />

<img src='../images/' />

# 

<img src='../images/' />

<img src='../images/' />

# 

<img src='../images/' />

<img src='../images/' />

# 

<img src='../images/' />

<img src='../images/' />

# 

<img src='../images/' />

<img src='../images/' />